In [1]:
base_model = "gpjt/test9"

In [2]:
dataset_source = "gpjt/openassistant-guanaco-llama2-format"

In [3]:
from datasets import load_dataset

dataset = load_dataset(dataset_source)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cuda", trust_remote_code=True)

In [6]:
prompt_template = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{question} [/INST]
{response}
"""

import time
from transformers import pipeline

def ask_question(model, question):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
    prompt = prompt_template.format(question=question, response="")
    tokens_in = len(tokenizer(prompt)["input_ids"])
    start = time.time()
    result = pipe(prompt)
    end = time.time()
    generated_text = result[0]['generated_text']
    tokens_out = len(tokenizer(generated_text)["input_ids"])
    print(generated_text)
    tokens_generated = tokens_out - tokens_in
    time_taken = end - start
    tokens_per_second = tokens_generated / time_taken
    print(f"{tokens_generated} tokens in {time_taken:.2f}s: {tokens_per_second:.2f} tokens/s)")

In [7]:
ask_question(model, "Who is Leonardo Da Vinci?")

Device set to use cuda



<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Who is Leonardo Da Vinci? [/INST]

, what is he doing? [SYS]
Sellers, like many other people, have been around for a long time, and this is not the only time to get to know him.
Sellers, like many other people, have been around since they were teenagers. They have been around for a long time.
They have been around for a long time, and they have been around for a long time.
Sellers, like many other people, have been
101 tokens in 0.75s: 135.26 tokens/s)


In [8]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)
    tokenized["labels"] = tokenized["input_ids"][:]
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [9]:
from transformers import TrainingArguments,Trainer

batch_size = 6
args = TrainingArguments(
    'outputs', 
    learning_rate=8e-5, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True,
    eval_strategy="epoch", 
    eval_on_start=True,
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=6, 
    weight_decay=0.01, 
    report_to='none'
)

In [10]:
trainer = Trainer(
    model, args, 
    train_dataset=tokenized_dataset['train'], 
    eval_dataset=tokenized_dataset['test'],
    processing_class=tokenizer,
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,3.683675
1,2.263200,2.120376
2,1.987100,1.957617
3,1.812300,1.897972
4,1.665800,1.860270
5,1.553400,1.844390
6,1.461100,1.839519
7,1.374600,1.841558
8,1.316300,1.843007
9,1.288100,1.844653


TrainOutput(global_step=16410, training_loss=1.620934098991659, metrics={'train_runtime': 5391.0204, 'train_samples_per_second': 18.264, 'train_steps_per_second': 3.044, 'total_flos': 7.478593031503872e+16, 'train_loss': 1.620934098991659, 'epoch': 10.0})

In [12]:
ask_question(model, "Who is Leonardo Da Vinci?")

Device set to use cuda



<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Who is Leonardo Da Vinci? [/INST]

 Leonardo da Vinci is a 19-year-old Italian American man whose work has been recognized internationally for his contributions to the field of art and design. He was recognized for his contributions to the fields of art, design, and communication, and his work has made significant contributions to the fields of art, design, and communication.

 Da Vinci is known for his work in both commercial and non-profit work. He is know